In [0]:
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
race_results_years_list = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_file_date}'") \
    .select("race_year") \
    .distinct() \
    .collect()

In [0]:
race_years = []
for element in race_results_years_list:
    race_years.append(element.race_year)

In [0]:
df_race_results = spark.read.format("delta").load(f"{presentation_folder_path}/race_results").filter(F.col("race_year").isin(race_years))

In [0]:
df_driver_standings = df_race_results.groupBy("race_year", "driver_name", "driver_nationality").agg(F.sum("points").alias("total_points"), F.count(F.when(F.col("position") == 1, True)).alias("wins"))

In [0]:
display(df_driver_standings.filter("race_year = 2020"))

In [0]:
from pyspark.sql import Window
driver_ranks_spec = Window.partitionBy("race_year").orderBy(F.col("total_points").desc(), F.col("wins").desc())

df_driver_standings = df_driver_standings.withColumn("rank", F.rank().over(driver_ranks_spec))\
    .withColumn("file_date", F.lit(v_file_date))

In [0]:
display(df_driver_standings.filter("race_year = 2020"))

In [0]:
# apply_incremental_load(df_driver_standings, "f1_presentation.driver_standings", "race_year")

merge_delta_data(df_driver_standings, 'f1_presentation','driver_standings',presentation_folder_path, 'tgt.race_year = src.race_year AND tgt.driver_name = src.driver_name','race_year')